In [138]:
import pandas as pd
import datetime
class sp500:

  def __init__(self):
    '''
    это если на колабе
    from google.colab import files
    files.upload()
    #файл называется s-p-500, я его залил на гитхаб
    '''
    self.df = pd.read_excel('s-p-500.xlsx')
    self.empty_series = self.df[self.df['Дата'] == '2021-12-31']['Значение']

  def is_correct(self, date):
    if len(date.split('-')) != 3:
      return False
    if not all([i.isdigit() for i in date.split('-')]):
      return False
    year, month, day = list(map(int, date.split('-')))
    if month == 2:
      if (year % 4 == 0 and year % 100 != 0) or year % 400 == 0:
        return day <= 29 and day > 0
    if month in [1, 3, 5, 7, 8, 10, 12]:
      return day <= 31 and day > 0
    return day <= 30 and day > 0

  def get(self, date):
    if not self.is_correct(date):
      print('Неправильная дата')
      return -1

    if int(date.split('-')[0]) > 2020 or int(date.split('-')[0]) < 2018:
      print('нет данных про s&p500 в этой дате')
      return -1
    
    if len(self.df[self.df['Дата'] == date]['Значение']) > 0:
      return self.df[self.df['Дата'] == date]['Значение']
    
    #в датасете нет некоторых дат, поэтому буду брать ближающую доступную
    a = list(map(int, date.split('-')))
    d = datetime.date(year = a[0], month = a[1], day = a[2])
    while len(self.df[self.df['Дата'] == f"{str(d).split('-')[0]}-{str(d).split('-')[1]}-{str(d).split('-')[2]}"]) == 0:
      d += datetime.timedelta(days = 1)
    return self.df[self.df['Дата'] == f"{str(d).split('-')[0]}-{str(d).split('-')[1]}-{str(d).split('-')[2]}"]['Значение']

  '''
  получение индекса по дате вида
  2020-12-31(год, месяц, день), если mode = 1,
  и если mode = 2(год, день, месяц), то
  по дате вида 2020-31-12
  '''
  def get_index_for_str(self, date, mode = 1):
    if mode == 1:
      return self.get(date)
    if mode == 2:
      a = date.split('-')
      new_date = f'{a[0]}-{a[2]}-{a[1]}'
      return self.get(new_date)    
  '''
  получения индекса по трем переменным даты(год, месяц, день)
  '''
  def get_index_for_date(self, year, month, day):
    if month < 10:
      month = '0' + str(month)
    if day < 10:
      day = '0' + str(day)
    return self.get(f'{year}-{month}-{day}')

In [139]:
test = sp500()

In [140]:
test.get_index_for_str('2020-12-31', mode = 1)

0    3756.07
Name: Значение, dtype: float64

In [141]:
test.get_index_for_str('2020-31-12', mode = 2)

0    3756.07
Name: Значение, dtype: float64

In [142]:
len(test.df[test.df['Дата'] == '2019-06-03'])

1

In [143]:
test.get_index_for_date(year = 2019, month = 6, day = 2)

405    2744.45
Name: Значение, dtype: float64

In [146]:
test.df[test.df['Дата'] == '2019-06-02']

,Дата,Значение
